### Exploring the US Immigration Data

Source: [US National Tourism and Trade Office](https://travel.trade.gov/research/reports/i94/historical/2016.html)

This data comes from the US National Tourism and Trade Office. A data dictionary is included in the workspace. There's a sample file so you can take a look at the data in csv format before reading it all in. You do not have to use the entire dataset, just use what you need to accomplish the goal you set at the beginning of the project.

In [4]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 50)

In [99]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as F
import pyspark.sql.types as T
from unidecode import unidecode
import re
import datetime as dt
from pyspark.sql import Row

In [2]:
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [5]:
df.limit(5).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,None,NaN,37.0,2.0,1.0,None,None,None,T,None,U,None,1979.0,10282016,None,None,None,1.897628e+09,None,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,25.0,3.0,1.0,20130811,SEO,None,G,None,Y,None,1991.0,D/S,M,None,None,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,55.0,2.0,1.0,20160401,None,None,T,O,None,M,1961.0,09302016,M,None,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,28.0,2.0,1.0,20160401,None,None,O,O,None,M,1988.0,09302016,None,None,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,4.0,2.0,1.0,20160401,None,None,O,O,None,M,2012.0,09302016,None,None,AA,9.246846e+10,00199,B2


In [6]:
df.count()

3096313

In [7]:
df.select("admnum").distinct().count()

3075579

In [12]:
temp = df.groupBy("admnum").count().filter(F.col("count")>1).limit(5).toPandas()

In [13]:
df.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [15]:
temp['admnum'] = temp['admnum'].astype(str) 

In [16]:
temp

,admnum,count
0,55442526433.0,2
1,55436862333.0,2
2,92497907430.0,2
3,85513120030.0,2
4,55498451333.0,2


In [19]:
df.filter(F.col("admnum")==55442526433).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,9733.0,2016.0,4.0,111.0,111.0,SEA,20545.0,1.0,AK,20554.0,54.0,2.0,1.0,20160401,None,None,G,K,None,M,1962.0,06292016,M,None,DL,5.544253e+10,00145,WT
1,2670912.0,2016.0,4.0,111.0,111.0,DAC,20559.0,3.0,None,20561.0,54.0,2.0,1.0,20160415,None,None,Z,O,None,M,1962.0,06292016,M,None,None,5.544253e+10,00145,WT


In [20]:
df.select("cicid").distinct().count()

3096313

#### Observations:

1. `admnum` is not unique to each row
2. `cicid` is unique for each row 

This is verified from the number of distinct `cicid`, `admnum` and `total number of rows`

In [21]:
df_2 =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_may16_sub.sas7bdat')

In [22]:
df_2.limit(5).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2.0,2016.0,5.0,207.0,207.0,XXX,20605.0,NaN,None,NaN,27.0,3.0,1.0,None,None,None,T,None,U,None,1989.0,D/S,None,None,None,1.141634e+09,None,F1
1,3.0,2016.0,5.0,209.0,209.0,XXX,20598.0,NaN,None,NaN,27.0,1.0,1.0,None,None,None,T,None,U,None,1989.0,05232018,None,None,None,1.863211e+09,None,E2
2,4.0,2016.0,5.0,213.0,213.0,XXX,20578.0,NaN,None,NaN,78.0,2.0,1.0,None,None,None,T,None,U,None,1938.0,11032016,None,None,None,4.696371e+09,None,B2
3,5.0,2016.0,5.0,213.0,213.0,XXX,20601.0,NaN,None,NaN,29.0,3.0,1.0,None,None,None,T,None,U,None,1987.0,D/S,None,None,None,1.141260e+09,None,F1
4,13.0,2016.0,5.0,213.0,213.0,CHI,20577.0,1.0,IL,20270.0,29.0,3.0,1.0,20150619,None,None,T,O,None,M,1987.0,D/S,F,None,EK,6.479287e+10,00235,F1


In [23]:
df_2.count()

3444249

In [24]:
df_2.select(F.col("cicid")).distinct().count()

3444249

In [25]:
df_2.select(F.col("admnum")).distinct().count()

3423255

In [26]:
df_merged = df.union(df_2)

In [27]:
df_merged.count()

6540562

In [28]:
3096313+3444249

6540562

In [29]:
df_merged.select(F.col("cicid")).distinct().count()

4929270

#### Observations:

1. `admnum` is not unique to each row
2. `cicid` is  unique for each row within a month 

This is verified from the number of distinct `cicid`, `admnum` and `total number of rows` for april and may months combined data.

#### Playing with the date columns

From here we will use sample data only:

date columns: `arrdate`, `depdate`, `dtaddto`

In [133]:
df_s = spark.read.options(delimiter=",", header="true")\
            .csv("immigration_data_sample.csv")

In [134]:
df_s.limit(5).toPandas()

,_c0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,None,None,G,O,None,M,1955.0,07202016,F,None,JL,56582674633.0,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,None,G,R,None,M,1990.0,10222016,M,None,*GA,94361995930.0,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,None,None,G,O,None,M,1940.0,07052016,M,None,LH,55780468433.0,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,None,G,O,None,M,1991.0,10272016,M,None,QR,94789696030.0,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,None,None,Z,K,None,M,1997.0,07042016,F,None,None,42322572633.0,LAND,WT


##### Filtering only for the Valid i94port codes

In [135]:
df_pc = spark.read.options(delimiter=",", header="true")\
            .csv("Cleaned Data/I94_ports_code.csv")

In [136]:
df_pc.limit(5).toPandas()

,code
0,CLG
1,EDA
2,YHC
3,HAL
4,MON


    Using the Broadcast join to filter out the invalid i94 port codes

In [137]:
df_sn = df_s.join(F.broadcast(df_pc), df_s.i94port==df_pc.code).drop("code")

In [138]:
df_sn.limit(5).toPandas()

,_c0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,None,None,G,O,None,M,1955.0,07202016,F,None,JL,56582674633.0,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,None,G,R,None,M,1990.0,10222016,M,None,*GA,94361995930.0,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,None,None,G,O,None,M,1940.0,07052016,M,None,LH,55780468433.0,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,None,G,O,None,M,1991.0,10272016,M,None,QR,94789696030.0,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,None,None,Z,K,None,M,1997.0,07042016,F,None,None,42322572633.0,LAND,WT


In [139]:
df_s.count()

1000

In [140]:
df_sn.count()

999

In [141]:
df_sn.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- cicid: string (nullable = true)
 |-- i94yr: string (nullable = true)
 |-- i94mon: string (nullable = true)
 |-- i94cit: string (nullable = true)
 |-- i94res: string (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: string (nullable = true)
 |-- i94mode: string (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: string (nullable = true)
 |-- i94bir: string (nullable = true)
 |-- i94visa: string (nullable = true)
 |-- count: string (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: string (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = tru

In [142]:
# get datetime from arrdate and depdate column value
get_date1 = F.udf(lambda x: (dt.datetime(1960, 1, 1).date() + dt.timedelta(x)).isoformat() if x else None, T.StringType())

In [143]:
df_sn = df_sn.withColumn('iso_arrdate', get_date1(F.col('arrdate').cast(T.DoubleType())))

In [144]:
df_sn.limit(5).toPandas()

,_c0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype,iso_arrdate
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,None,None,G,O,None,M,1955.0,07202016,F,None,JL,56582674633.0,00782,WT,2016-04-22
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,None,G,R,None,M,1990.0,10222016,M,None,*GA,94361995930.0,XBLNG,B2,2016-04-23
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,None,None,G,O,None,M,1940.0,07052016,M,None,LH,55780468433.0,00464,WT,2016-04-07
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,None,G,O,None,M,1991.0,10272016,M,None,QR,94789696030.0,00739,B2,2016-04-28
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,None,None,Z,K,None,M,1997.0,07042016,F,None,None,42322572633.0,LAND,WT,2016-04-06


In [145]:
df_sn.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- cicid: string (nullable = true)
 |-- i94yr: string (nullable = true)
 |-- i94mon: string (nullable = true)
 |-- i94cit: string (nullable = true)
 |-- i94res: string (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: string (nullable = true)
 |-- i94mode: string (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: string (nullable = true)
 |-- i94bir: string (nullable = true)
 |-- i94visa: string (nullable = true)
 |-- count: string (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: string (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = tru

In [146]:
df_sn = df_sn.withColumn('iso_depdate', get_date1(F.col('depdate').cast(T.DoubleType())))

In [147]:
df_sn.limit(5).toPandas()

,_c0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype,iso_arrdate,iso_depdate
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,None,None,G,O,None,M,1955.0,07202016,F,None,JL,56582674633.0,00782,WT,2016-04-22,2016-04-29
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,None,G,R,None,M,1990.0,10222016,M,None,*GA,94361995930.0,XBLNG,B2,2016-04-23,2016-04-24
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,None,None,G,O,None,M,1940.0,07052016,M,None,LH,55780468433.0,00464,WT,2016-04-07,2016-04-27
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,None,G,O,None,M,1991.0,10272016,M,None,QR,94789696030.0,00739,B2,2016-04-28,2016-05-07
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,None,None,Z,K,None,M,1997.0,07042016,F,None,None,42322572633.0,LAND,WT,2016-04-06,2016-04-09


In [148]:
get_date2 = F.udf(lambda x: x[4:]+'-'+x[:2]+'-'+x[2:4], T.StringType())

In [149]:
df_sn = df_sn.withColumn('iso_duedate', get_date2(F.col('dtaddto')))

In [150]:
df_sn.limit(5).toPandas()

,_c0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype,iso_arrdate,iso_depdate,iso_duedate
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,None,None,G,O,None,M,1955.0,07202016,F,None,JL,56582674633.0,00782,WT,2016-04-22,2016-04-29,2016-07-20
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,None,G,R,None,M,1990.0,10222016,M,None,*GA,94361995930.0,XBLNG,B2,2016-04-23,2016-04-24,2016-10-22
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,None,None,G,O,None,M,1940.0,07052016,M,None,LH,55780468433.0,00464,WT,2016-04-07,2016-04-27,2016-07-05
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,None,G,O,None,M,1991.0,10272016,M,None,QR,94789696030.0,00739,B2,2016-04-28,2016-05-07,2016-10-27
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,None,None,Z,K,None,M,1997.0,07042016,F,None,None,42322572633.0,LAND,WT,2016-04-06,2016-04-09,2016-07-04


In [151]:
i94mode = [('1.0', 'Air'),('2.0','Sea'),('3.0','Land'),('9.0','Not Reported')]
i94mode_rdd = spark.sparkContext.parallelize(i94mode).map(lambda x: Row(i94mode=x[0], i94_mode=x[1]))
i94mode_df = spark.createDataFrame(i94mode_rdd)

In [152]:
i94mode_df.show()

+------------+-------+
|    i94_mode|i94mode|
+------------+-------+
|         Air|    1.0|
|         Sea|    2.0|
|        Land|    3.0|
|Not Reported|    9.0|
+------------+-------+



In [153]:
df_sn = df_sn.join(F.broadcast(i94mode_df), df_sn.i94mode==i94mode_df.i94mode, 'left')

In [154]:
df_sn.limit(5).toPandas()

,_c0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype,iso_arrdate,iso_depdate,iso_duedate,i94_mode,i94mode
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,None,None,G,O,None,M,1955.0,07202016,F,None,JL,56582674633.0,00782,WT,2016-04-22,2016-04-29,2016-07-20,Air,1.0
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,None,G,R,None,M,1990.0,10222016,M,None,*GA,94361995930.0,XBLNG,B2,2016-04-23,2016-04-24,2016-10-22,Air,1.0
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,None,None,G,O,None,M,1940.0,07052016,M,None,LH,55780468433.0,00464,WT,2016-04-07,2016-04-27,2016-07-05,Air,1.0
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,None,G,O,None,M,1991.0,10272016,M,None,QR,94789696030.0,00739,B2,2016-04-28,2016-05-07,2016-10-27,Air,1.0
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,None,None,Z,K,None,M,1997.0,07042016,F,None,None,42322572633.0,LAND,WT,2016-04-06,2016-04-09,2016-07-04,Land,3.0


In [157]:
i94visa = [('1.0', 'Business'),('2.0','Pleasure'),('3.0','Student')]
i94visa_rdd = spark.sparkContext.parallelize(i94visa).map(lambda x: Row(i94visa=x[0], i94_visa=x[1]))
i94visa_df = spark.createDataFrame(i94visa_rdd)

In [159]:
i94visa_df.show()

+--------+-------+
|i94_visa|i94visa|
+--------+-------+
|Business|    1.0|
|Pleasure|    2.0|
| Student|    3.0|
+--------+-------+



In [160]:
df_sn = df_sn.join(F.broadcast(i94visa_df), df_sn.i94visa==i94visa_df.i94visa, 'left')

In [161]:
df_sn.limit(5).toPandas()

,_c0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype,iso_arrdate,iso_depdate,iso_duedate,i94_mode,i94mode,i94_visa,i94visa
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,None,None,G,O,None,M,1955.0,07202016,F,None,JL,56582674633.0,00782,WT,2016-04-22,2016-04-29,2016-07-20,Air,1.0,Pleasure,2.0
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,None,G,R,None,M,1990.0,10222016,M,None,*GA,94361995930.0,XBLNG,B2,2016-04-23,2016-04-24,2016-10-22,Air,1.0,Pleasure,2.0
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,None,None,G,O,None,M,1940.0,07052016,M,None,LH,55780468433.0,00464,WT,2016-04-07,2016-04-27,2016-07-05,Air,1.0,Pleasure,2.0
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,None,G,O,None,M,1991.0,10272016,M,None,QR,94789696030.0,00739,B2,2016-04-28,2016-05-07,2016-10-27,Air,1.0,Pleasure,2.0
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,None,None,Z,K,None,M,1997.0,07042016,F,None,None,42322572633.0,LAND,WT,2016-04-06,2016-04-09,2016-07-04,Land,3.0,Pleasure,2.0


In [162]:
df_i94addr = spark.read.options(delimiter=",", header=True)\
                    .csv("Cleaned Data/I94_addr.csv")

In [163]:
df_i94addr.limit(5).toPandas()

,i94addr,i94addr_US_state
0,AL,Alabama
1,AK,Alaska
2,AZ,Arizona
3,AR,Arkansas
4,CA,California


In [164]:
df_sn = df_sn.join(F.broadcast(df_i94addr), df_sn.i94addr==df_i94addr.i94addr, "left")

In [165]:
df_sn.limit(5).toPandas()

,_c0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype,iso_arrdate,iso_depdate,iso_duedate,i94_mode,i94mode,i94_visa,i94visa,i94addr,i94addr_US_state
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,None,None,G,O,None,M,1955.0,07202016,F,None,JL,56582674633.0,00782,WT,2016-04-22,2016-04-29,2016-07-20,Air,1.0,Pleasure,2.0,HI,Hawaii
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,None,G,R,None,M,1990.0,10222016,M,None,*GA,94361995930.0,XBLNG,B2,2016-04-23,2016-04-24,2016-10-22,Air,1.0,Pleasure,2.0,TX,Texas
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,None,None,G,O,None,M,1940.0,07052016,M,None,LH,55780468433.0,00464,WT,2016-04-07,2016-04-27,2016-07-05,Air,1.0,Pleasure,2.0,FL,Florida
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,None,G,O,None,M,1991.0,10272016,M,None,QR,94789696030.0,00739,B2,2016-04-28,2016-05-07,2016-10-27,Air,1.0,Pleasure,2.0,CA,California
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,None,None,Z,K,None,M,1997.0,07042016,F,None,None,42322572633.0,LAND,WT,2016-04-06,2016-04-09,2016-07-04,Land,3.0,Pleasure,2.0,NY,New York


In [170]:
df_f = df_sn.select('cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port', 'iso_arrdate', 'iso_depdate', 'iso_duedate', \
                 'i94_visa', 'i94_mode', 'admnum', 'insnum', 'i94addr_US_state', 'airline', 'fltno', 'visatype', 'i94bir', 'gender')

In [171]:
df_f.limit(5).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,iso_arrdate,iso_depdate,iso_duedate,i94_visa,i94_mode,admnum,insnum,i94addr_US_state,airline,fltno,visatype,i94bir,gender
0,4084316.0,2016.0,4.0,209.0,209.0,HHW,2016-04-22,2016-04-29,2016-07-20,Pleasure,Air,56582674633.0,None,Hawaii,JL,00782,WT,61.0,F
1,4422636.0,2016.0,4.0,582.0,582.0,MCA,2016-04-23,2016-04-24,2016-10-22,Pleasure,Air,94361995930.0,None,Texas,*GA,XBLNG,B2,26.0,M
2,1195600.0,2016.0,4.0,148.0,112.0,OGG,2016-04-07,2016-04-27,2016-07-05,Pleasure,Air,55780468433.0,None,Florida,LH,00464,WT,76.0,M
3,5291768.0,2016.0,4.0,297.0,297.0,LOS,2016-04-28,2016-05-07,2016-10-27,Pleasure,Air,94789696030.0,None,California,QR,00739,B2,25.0,M
4,985523.0,2016.0,4.0,111.0,111.0,CHM,2016-04-06,2016-04-09,2016-07-04,Pleasure,Land,42322572633.0,None,New York,None,LAND,WT,19.0,F
